In [ ]:
#!pip install --upgrade pandas_profiling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
import gc
import lightgbm as lgb
import module.train_log as log
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from keras.callbacks import EarlyStopping
from numba import jit
from tqdm import tqdm_notebook as tqdm
import seaborn as sns 
from datetime import datetime as dt
import pandas_profiling as pp
%matplotlib inline

In [3]:
@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', fast_auc(labels, preds), True

In [4]:
# 불러올 데이터의 타입과 컬럼을 정의한다.
def get_column_dtypes():
    return {
        'MachineIdentifier':                                    'object',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'}

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
def get_type_columns():
    dtypes = get_column_dtypes()
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerical_columns = [c for c,v in dtypes.items() if v in numerics]
    categorical_columns = [c for c,v in dtypes.items() if v not in numerics]
    return numerical_columns, categorical_columns

In [6]:
# 데이터를 불러온다.
def load_data():
    column_dtypes = get_column_dtypes()
    #use_columns = get_use_columns()
    tr_train = pd.read_csv('../input/train.csv', dtype=column_dtypes)
    X_test = pd.read_csv('../input/test.csv', dtype=column_dtypes)
    
    # 학습 데이터의 xy를 분리한다.
    y_train = tr_train.HasDetections
    X_train = tr_train.drop(['HasDetections'], axis=1)

    X_train.to_pickle("../result/X_train.pkl")
    y_train.to_pickle("../result/y_train.pkl")
    X_test.to_pickle("../result/X_test.pkl")
    return X_train, y_train, X_test


In [7]:
# 데이터를 불러온다.
def load_pickle_data(data_type:str=None):
    global X_train, y_train, X_test, ID_test
    if data_type is not None:
        X_train_name = 'X_train_{}'.format(data_type)
        X_test_name = 'X_test_{}'.format(data_type)
        ID_test_name = 'ID_test_{}'.format(data_type)
    else:
        X_train_name = 'X_train'
        X_test_name = 'X_test'
    
    X_train = pd.read_pickle("../result/{}.pkl".format(X_train_name))
    print("Loaded X_train")
    y_train = pd.read_pickle("../result/y_train.pkl")
    print("Loaded y_train")
    X_test = pd.read_pickle("../result/{}.pkl".format(X_test_name))
    print("Loaded X_test")
    if data_type == 'scaled':
        ID_test = pd.read_pickle("../result/{}.pkl".format(ID_test_name))
        print("Loaded ID_test")
    return X_train, y_train, X_test


def get_sample(n:int=1000):
    global X_train, y_train
    X_train = X_train.sample(n=n) 
    y_train = y_train[X_train.index] 
    return X_train, y_train

def get_sample_include_ID(n:int=1000):
    global X_train, y_train
    X_train = X_train.sample(n=n) 
    y_train = y_train[X_train.index] 
    return X_train, y_train

In [8]:
# Target Encoder를 실행한다.
def to_target_encoder():
    global X_train, X_test
    category_columns = list(X_train.select_dtypes(include=['category']).columns)
    print("Columns to encode:", category_columns)
    enc = ce.TargetEncoder(cols=category_columns)
    enc.fit(X_train, y_train)
    print("Fitted")
    X_train = enc.transform(X_train)
    print("Transformed train data")
    X_test = enc.transform(X_test)
    print("Transformed test data")
    
    X_train.to_pickle("../result/X_train_target_encoded.pkl")
    X_test.to_pickle("../result/X_test_target_encoded.pkl")
    print("Saved to pickle data")
    return X_train, X_test

In [9]:
# 결측치를 평균값으로 채운다.
def fill_na():
    for column in list(X_train.columns[X_train.isna().any()]):
        if  pd.api.types.is_categorical_dtype(X_train[column]):
            X_train[column] = X_train[column].cat.add_categories(['NA'])
            X_train[column].fillna('NA', inplace=True)
            X_test[column] = X_test[column].cat.add_categories(['NA'])
            X_test[column].fillna('NA', inplace=True)
        else:
            mean = X_train[column].mean()
            X_train[column].fillna(mean, inplace=True)
            X_test[column].fillna(mean, inplace=True)

    X_train.to_pickle("../result/X_train_filled_na.pkl")
    X_test.to_pickle("../result/X_test_filled_na.pkl")
    return X_train, X_test

In [10]:
# ID 저장 및 제거 
def drop_id_n_get_test_id():
    ID_test = X_test.MachineIdentifier
    X_train.drop(['MachineIdentifier'], axis=1, inplace=True)
    X_test.drop(['MachineIdentifier'], axis=1, inplace=True)
    return ID_test

In [11]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

def get_model_mlp(optimizer='adam', init='glorot_uniform'):
    input_dim = X_train.columns.size
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim, activation='relu', kernel_initializer=init))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu', kernel_initializer=init))
    model.add(Dropout(0.2))
    model.add(Dense(64,activation='relu', kernel_initializer=init))
    model.add(Dropout(0.2))
    model.add(Dense(32,activation='relu', kernel_initializer=init))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid',kernel_initializer=init))
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=[auroc])
    return model

In [12]:
def get_grid_params(seed):
    return {
         'lgb__num_leaves': [60],
         'lgb__min_data_in_leaf': [60], 
         'lgb__objective':['binary'],
         'lgb__max_depth': [-1],
         'lgb__learning_rate': [0.05],
         "lgb__boosting": ["gbdt"],
         "lgb__feature_fraction": [0.8],
         "lgb__bagging_freq": [1],
         "lgb__bagging_fraction": [0.8] ,
         "lgb__bagging_seed": [11],
         "lgb__metric": ['auc'],
         "lgb__lambda_l1": [0.1],
         "lgb__random_state": [seed],
         "lgb__n_estimators": [40],
#          "mlp__optimizer": ['adam'],
#          "mlp__init": ['uniform'],
#          "mlp__epochs": [20],
#          "mlp__batch_size": [256]
    }
def get_base_params(random_state):
    return {   
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'nthread': 4,
        'learning_rate': 0.05,
        'max_depth': 5,
        'num_leaves': 40,
        'sub_feature': 0.9,
        'sub_row':0.9,
        'bagging_freq': 1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'random_state': random_state
        }

In [13]:
def trains(X_train, y_train, X_test):
    N_FOLD = 5
    SEED = 42
    
    folds = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)
    kf = folds.split(X=X_train, y=y_train)
    grid_params = get_grid_params(SEED)
    clf_lgb = LGBMClassifier()
    clf_mlp = KerasClassifier(build_fn=get_model_mlp, verbose=1)
    
    clfs = [
        ('lgb', clf_lgb),
#         ('mlp', clf_mlp),
    ]
    clf_eb = VotingClassifier(estimators=clfs, voting='soft')
    
    clf_cv = GridSearchCV(estimator=clf_eb, param_grid=grid_params, cv=kf, verbose=1)
    clf_cv.fit(X_train, y_train)
    
    log.save("southman lgb", X_train, clfs, clf_cv)
    print(clf_cv.best_params_, clf_cv.best_score_)
    #clf_cv.save_model('../result/lgb_model.txt')    
    
    return clf_cv.predict_proba(X_test)[:, 1] 

In [14]:
def train_model(x, y, lgb_params, 
                number_of_folds=5, 
                evaluation_metric='auc', 
                save_feature_importances=False, 
                early_stopping_rounds=50, 
                num_round = 50,
                random_state = 42,
                shuffle = True,
                break_first=False):
    cross_validator = StratifiedKFold(n_splits=number_of_folds,
                                  random_state=random_state,
                                  shuffle=shuffle)
    
    validation_scores = []
    classifier_models = []
    feature_importance_df = pd.DataFrame()
    for fold_index, (train_index, validation_index) in enumerate(cross_validator.split(x, y)):
        gc.collect()
        x_train, x_validation = x.iloc[train_index], x.iloc[validation_index]
        y_train, y_validation = y.iloc[train_index], y.iloc[validation_index]
    
        x_train_columns = x_train.columns
        trn_data = lgb.Dataset(x_train, label=y_train)
        val_data = lgb.Dataset(x_validation, label=y_validation)
        classifier_model = lgb.train(lgb_params,
                                     trn_data,
                                     num_round,
                                     valid_sets=[trn_data, val_data],
                                     verbose_eval=100,
                                     early_stopping_rounds=early_stopping_rounds,
                                     feval=eval_auc
                                     )

        classifier_models.append(classifier_model)
        
        predictions = classifier_model.predict(x_validation, num_iteration=classifier_model.best_iteration)
        false_positive_rate, recall, thresholds = metrics.roc_curve(y_validation, predictions)
        score = metrics.auc(false_positive_rate, recall)
        validation_scores.append(score)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = x_train_columns
        fold_importance_df["importance"] = classifier_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = fold_index + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        if break_first:
            break
        
    if save_feature_importances:
        cols = (feature_importance_df[["feature", "importance"]]
                .groupby("feature")
                .mean()
                .sort_values(by="importance", ascending=False)[:1000].index)

        best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

        plt.figure(figsize=(14, 25))
        sns.barplot(x="importance",
                    y="feature",
                    data=best_features.sort_values(by="importance",
                                                   ascending=False))
        plt.title('LightGBM Features (avg over folds)')
        plt.tight_layout()
        plt.savefig('lgbm_importances.png')

        # mean_gain = feature_importances[['gain', 'feature']].groupby('feature').mean()
        # feature_importances['mean_gain'] = feature_importances['feature'].map(mean_gain['gain'])
        #
        # temp = feature_importances.sort_values('mean_gain', ascending=False)
        best_features.sort_values(by="importance", ascending=False) \
            .groupby("feature") \
            .mean() \
            .sort_values(by="importance", ascending=False) \
            .to_csv('feature_importances_new.csv', index=True)

    score = sum(validation_scores) / len(validation_scores)
    return classifier_models, score

In [15]:
def get_predicts(models, X_test):
    models_cnt = len(models)
    predictions = np.zeros(len(X_test))
    for model in tqdm(models):
        current_pred = model.predict(X_test, num_iteration=model.best_iteration)
        predictions += current_pred / models_cnt
    return predictions

In [16]:
def to_scale():
    global X_train, X_test, ID_test
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X_train)
    print("Fitted to scale")
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
    print("Scaled train data")
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    print("Scaled test data")
    X_train.to_pickle("../result/X_train_scaled.pkl")
    X_test.to_pickle("../result/X_test_scaled.pkl")
    ID_test.to_pickle("../result/ID_test_scaled.pkl")
    print("Saved to pickle data")
                

In [17]:
def to_categorical(features):
    global X_train, X_test
    for feature in tqdm(features):
        X_train[feature] = X_train[feature].astype('category')
        X_test[feature] = X_test[feature].astype('category')

def to_numberic(X, feature):
    if X[feature].dtype != 'int32' :
        X[feature] = X[feature].astype('str')
        X.loc[X[feature] == "NA", feature] = "0"
    X[feature] = round(X[feature].astype('float')).astype('int32')

def to_numberics(features):
    global X_train, X_test
    for feature in tqdm(features):
        to_numberic(X_train, feature)
        to_numberic(X_test, feature)

In [18]:
def encode_categorical_columns(x_train, x_test, columns, sort=True):
    train_length = x_train.shape[0]
    for col in tqdm(columns):
        if col == 'MachineIdentifier' or col == 'HasDetections':
            continue
            
        combined_data = pd.concat([x_train[col], x_test[col]])
        combined_data, _ = pd.factorize(combined_data, sort=sort)
        combined_data = pd.Series(combined_data).astype('int32')
        x_train[col] = combined_data.iloc[:train_length].values
        x_test[col] = combined_data.iloc[train_length:].values
        x_train[col] = x_train[col].fillna(0)
        x_test[col] = x_test[col].fillna(0)
        del combined_data
        
    return x_train, x_test

In [19]:
def remove_features(features, X_train, X_test):

    for feature in tqdm(features):
        if feature in X_train.columns:
            X_train.drop(feature, axis=1, inplace=True)
            X_test.drop(feature, axis=1, inplace=True)

In [20]:
def add_feature_engine_version(X):
    X['EngineVersion_1'] = X.EngineVersion.str.split(".", expand=True)[2].astype('int16')
    X['EngineVersion_2'] = X.EngineVersion.str.split(".", expand=True)[3].astype('int8')

def add_feature_app_version(X):
    X['AppVersion_1'] = X.AppVersion.str.split(".", expand=True)[1].astype('int8')
    X['AppVersion_2'] = X.AppVersion.str.split(".", expand=True)[2].astype('int16')
    X['AppVersion_3'] = X.AppVersion.str.split(".", expand=True)[3].astype('int16')

def add_feature_av_sig_version(X):
    X['AvSigVersion_1'] = X.AvSigVersion.str.split(".", expand=True)[0].astype('int8')
    X['AvSigVersion_2'] = X.AvSigVersion.str.split(".", expand=True)[1]
    X.loc[X.AvSigVersion_2 == '2&#x17;3', 'AvSigVersion_2'] = '23'
    X['AvSigVersion_2'] = X.AvSigVersion_2.astype('int16')
    X['AvSigVersion_3'] = X.AvSigVersion.str.split(".", expand=True)[2].astype('int16')
    
def add_feature_os_ver(X):
    X['OsVer_1'] = X.OsVer.str.split(".", expand=True)[0].astype('int8')
    X['OsVer_2'] = X.OsVer.str.split(".", expand=True)[1].astype('int8')
    X['OsVer_3'] = X.OsVer.str.split(".", expand=True)[2].astype('int16')
    X['OsVer_4'] = X.OsVer.str.split(".", expand=True)[3].astype('int16')
    
def add_feature_os_build_lab(X):
    X['OsBuildLab_1'] = X.OsBuildLab.str.split(".", expand=True)[0]
    X.loc[X.OsBuildLab_1.isna(), 'OsBuildLab_1'] = '0'
    X['OsBuildLab_1'] = X.OsBuildLab_1.astype('int16')
    X['OsBuildLab_2'] = X.OsBuildLab.str.split(".", expand=True)[1]
    X.loc[X.OsBuildLab_2.isna(), 'OsBuildLab_2'] = '0'
    X.loc[X.OsBuildLab_2 == '1*amd64fre', 'OsBuildLab_2'] = '64'
    X['OsBuildLab_2'] = X.OsBuildLab_2.astype('int16')
    X['OsBuildLab_3'] = X.OsBuildLab.str.split(".", expand=True)[2].astype('category')
    X['OsBuildLab_4'] = X.OsBuildLab.str.split(".", expand=True)[3].astype('category')
    X['OsBuildLab_5'] = X.OsBuildLab.str.split(".", expand=True)[4].astype('category')
    
def add_feature_c_display_Inches(X):
    feature = 'Census_InternalPrimaryDiagonalDisplaySizeInInches_round'
    X[feature] = round(X['Census_InternalPrimaryDiagonalDisplaySizeInInches'])
    X.loc[X[feature].isna() ,feature] = 0
    X[feature] = X[feature].astype('int16')

def add_feature_c_os_version(X):
    X['Census_OSVersion_1'] = X.Census_OSVersion.str.split(".", expand=True)[0].astype('int8')
    X['Census_OSVersion_2'] = X.Census_OSVersion.str.split(".", expand=True)[1].astype('int8')
    X['Census_OSVersion_3'] = X.Census_OSVersion.str.split(".", expand=True)[2].astype('int16')
    X['Census_OSVersion_4'] = X.Census_OSVersion.str.split(".", expand=True)[3].astype('int16')
    
def add_feature_c_os_branch(X):
    X['Census_OSBranch_1'] = X.Census_OSBranch.str.split("_", expand=True)[0].astype('category')
    X['Census_OSBranch_2'] = X.Census_OSBranch.str.split("_", expand=True)[1].astype('category')

In [21]:
# https://www.kaggle.com/artgor/is-this-malware-eda-fe-and-lgb-updated
def add_feature_primary_drive_c_ratio(X):
    X['primary_drive_c_ratio'] = X['Census_SystemVolumeTotalCapacity']/ X['Census_PrimaryDiskTotalCapacity']
    
def add_feature_non_primary_drive(X):
    X['non_primary_drive_MB'] = X['Census_PrimaryDiskTotalCapacity'] - X['Census_SystemVolumeTotalCapacity']

def add_feature_aspect_ratio(X):
    X['aspect_ratio'] = X['Census_InternalPrimaryDisplayResolutionHorizontal']/ X['Census_InternalPrimaryDisplayResolutionVertical']

def add_feature_monitor_dims(X):
    X['monitor_dims'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype(str) + '*' + X['Census_InternalPrimaryDisplayResolutionVertical'].astype('str')
    X['monitor_dims'] = X['monitor_dims'].astype('category')
    
def add_feature_dpi(X):
    X['Census_InternalPrimaryDisplayResolutionHorizontal'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype("float32")
    X['Census_InternalPrimaryDisplayResolutionVertical'] = X['Census_InternalPrimaryDisplayResolutionVertical'].astype("float32")
    X['Census_InternalPrimaryDiagonalDisplaySizeInInches'] = X['Census_InternalPrimaryDiagonalDisplaySizeInInches'].astype("float32")
    
    X['dpi'] = ((X['Census_InternalPrimaryDisplayResolutionHorizontal']**2 + X['Census_InternalPrimaryDisplayResolutionVertical']**2)**.5)/(X['Census_InternalPrimaryDiagonalDisplaySizeInInches'])
    X['dpi_square'] = X['dpi'] ** 2
    
def add_feature_mega_pixels(X):
    X['Census_InternalPrimaryDisplayResolutionHorizontal'] = X['Census_InternalPrimaryDisplayResolutionHorizontal'].astype("float32")
    X['Census_InternalPrimaryDisplayResolutionVertical'] = X['Census_InternalPrimaryDisplayResolutionVertical'].astype("float32")
    
    X['MegaPixels'] = (X['Census_InternalPrimaryDisplayResolutionHorizontal'] * X['Census_InternalPrimaryDisplayResolutionVertical'])/1e6


def add_feature_screen_area(X):
    X['Screen_Area'] = (X['aspect_ratio']* (X['Census_InternalPrimaryDiagonalDisplaySizeInInches']**2))/(X['aspect_ratio']**2 + 1)
    
def add_feature_ram_per_processor(X):
    X['ram_per_processor'] = X['Census_TotalPhysicalRAM']/ X['Census_ProcessorCoreCount']
    
def add_feature_new_num(X):
    X['new_num_0'] = X['Census_InternalPrimaryDiagonalDisplaySizeInInches'] / X['Census_ProcessorCoreCount']
    X['new_num_1'] = X['Census_ProcessorCoreCount'] * X['Census_InternalPrimaryDiagonalDisplaySizeInInches']

In [ ]:
%%time
X_train, y_train, X_test = load_data()

In [ ]:
# to_categorical(['DefaultBrowsersIdentifier', 'AVProductStatesIdentifier', 'CountryIdentifier', 
#               'CityIdentifier', 'OrganizationIdentifier', 'GeoNameIdentifier', 
#               'LocaleEnglishNameIdentifier', 'OsSuite', 'IeVerIdentifier', 'UacLuaenable',
#               'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier', 
#               'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorModelIdentifier', 
#               'Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier', 
#               'Wdft_RegionIdentifier'])

In [ ]:
add_feature_methods = [add_feature_engine_version, add_feature_app_version, add_feature_av_sig_version, add_feature_os_ver,
                       add_feature_os_build_lab, add_feature_c_display_Inches, add_feature_c_os_version, add_feature_c_os_branch,
                       add_feature_primary_drive_c_ratio, add_feature_non_primary_drive, add_feature_aspect_ratio,
                       add_feature_monitor_dims, add_feature_dpi, add_feature_mega_pixels, add_feature_screen_area,
                       add_feature_ram_per_processor, add_feature_new_num]

for method in tqdm(add_feature_methods):
    method(X_train)
    method(X_test)

In [ ]:
X_train[1:5]

In [ ]:
bad_cols = []
for col in tqdm(X_train.columns):
    rate = X_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate < 0.1:
        bad_cols.append(col)
        
for feature in tqdm(bad_cols):
    X_train.drop(feature, axis=1, inplace=True)
    X_test.drop(feature, axis=1, inplace=True)

In [ ]:
#categorical_columns = list(X_train.select_dtypes(include=['category']).columns)
#encode_categorical_columns(X_train, X_test, categorical_columns)

In [ ]:
remove_feature = ['OsVer_2', 'Census_OSVersion_1', 'Census_OSVersion_2', 'OsVer_1', 'AvSigVersion_1',
                  'IsBeta', 'AutoSampleOptIn', 'Census_ProcessorClass', 'Census_InternalBatteryType', 
                  'UacLuaenable', 'Census_OSArchitecture', 'Census_OSBuildNumber', 'OsPlatformSubRelease', 
                  'Census_IsFlightingInternal', 'Census_ProcessorCoreCount', 'Census_ThresholdOptIn', 
                  'Census_IsAlwaysOnAlwaysConnectedCapable', 'Census_OSBranch_1', 'HasTpm', 
                  'Census_InternalPrimaryDiagonalDisplaySizeInInches_round', 'Census_IsPenCapable', 
                  'Census_IsWIMBootEnabled', 'PuaMode', 'Census_IsFlightsDisabled', 'Census_OSBranch_2', 
                  'Census_IsPortableOperatingSystem', 'Platform', 'Census_ProcessorManufacturerIdentifier', 
                  'Census_OSVersion_3', 'ProductName', 'OsVer_3', 'OsVer', 'Census_DeviceFamily', 'OsVer_4']

remove_features(remove_feature, X_train, X_test)

In [35]:
remove_feature2 = ['Census_PowerPlatformRoleName', 'Census_FlightRing', 'SkuEdition', 'IeVerIdentifier', 
                  'AppVersion_1', 'Firewall', 'Census_IsSecureBootEnabled', 'ram_per_processor', 
                  'Census_OSBuildRevision', 'Census_OSBranch', 'OsSuite', 'OsBuildLab_3', 'Census_IsTouchEnabled', 
                  'OsBuildLab_2', 'aspect_ratio', 'OsBuild', 'Census_InternalPrimaryDisplayResolutionHorizontal', 'EngineVersion_2']

remove_features(remove_feature2, X_train, X_test)

In [38]:
remove_feature3 = ['AvSigVersion_2', 'Census_InternalPrimaryDisplayResolutionVertical', 'Census_OSUILocaleIdentifier', 
                 'Census_OSVersion_4', 'MegaPixels', 'OsBuildLab_1', 'Screen_Area', 'dpi_square']

remove_features(remove_feature3, X_train, X_test)

In [30]:
%%time
reduce_mem_usage(X_train)
reduce_mem_usage(X_test)

Mem. usage decreased to 842.31 Mb (52.2% reduction)
Mem. usage decreased to 696.52 Mb (53.7% reduction)
Wall time: 14.6 s


In [28]:
len(X_train.columns)

46

In [ ]:
gc.collect()
pp.ProfileReport(X_train)

In [ ]:
log_feature =['Census_PrimaryDiskTotalCapacity', 'Census_TotalPhysicalRAM']

for feature in tqdm(log_feature):
    X_train[feature] = np.log(X_train[feature])
    X_test[feature] = np.log(X_test[feature])

In [31]:
%%time
X_train.to_pickle("../result/X_train_20190130.pkl")
X_test.to_pickle("../result/X_test_20190130.pkl")
#X_train, y_train, X_test = load_pickle_data('20190129')
#ID_test = pd.read_pickle("../result/ID_test.pkl")

Wall time: 12.9 s


In [ ]:
#%%time
#ID_test = drop_id_n_get_test_id()

In [23]:
#X_train, y_train = get_sample(100000)

In [32]:
X_train.dtypes

EngineVersion                                        float16
AppVersion                                           float16
RtpStateBitfield                                     float16
IsSxsPassiveMode                                        int8
DefaultBrowsersIdentifier                            float16
AVProductStatesIdentifier                            float32
AVProductsInstalled                                  float16
AVProductsEnabled                                    float16
OrganizationIdentifier                               float16
GeoNameIdentifier                                    float16
LocaleEnglishNameIdentifier                             int8
Processor                                            float16
OsBuildLab                                           float16
IsProtected                                          float16
SMode                                                float16
SmartScreen                                          float16
Census_MDC2FormFactor   

In [27]:
to_target_encoder()

Columns to encode: ['EngineVersion', 'AppVersion', 'Processor', 'OsBuildLab', 'SmartScreen', 'Census_MDC2FormFactor', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_OSVersion', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel', 'OsBuildLab_4', 'OsBuildLab_5', 'monitor_dims']
Fitted
Transformed train data
Transformed test data
Saved to pickle data


(         EngineVersion  AppVersion  RtpStateBitfield  IsSxsPassiveMode  \
 0             0.552693    0.530385               7.0                 0   
 1             0.437158    0.521794               7.0                 0   
 2             0.552693    0.530385               7.0                 0   
 3             0.552693    0.530385               7.0                 0   
 4             0.552693    0.530385               7.0                 0   
 5             0.552693    0.530385               7.0                 0   
 6             0.552693    0.530385               7.0                 0   
 7             0.552693    0.530385               7.0                 0   
 8             0.491732    0.530385               7.0                 0   
 9             0.552693    0.530385               7.0                 0   
 10            0.552693    0.530385               7.0                 0   
 11            0.552693    0.530385               7.0                 0   
 12            0.552693  

In [ ]:
%%time
gc.collect()
SEED = 42
KFOLD = 2
models, validation_score = train_model(X_train, y_train, get_base_params(SEED),
                                      num_round=10000,
                                      early_stopping_rounds=10,
                                      save_feature_importances=True,
                                      random_state=SEED,
                                      number_of_folds = KFOLD,
                                      break_first=False)

Training until validation scores don't improve for 10 rounds.
[100]	training's auc: 0.701342	training's auc: 0.701342	valid_1's auc: 0.700602	valid_1's auc: 0.700602
[200]	training's auc: 0.708981	training's auc: 0.708981	valid_1's auc: 0.707958	valid_1's auc: 0.707958
[300]	training's auc: 0.712934	training's auc: 0.712934	valid_1's auc: 0.711672	valid_1's auc: 0.711672
[400]	training's auc: 0.715609	training's auc: 0.715609	valid_1's auc: 0.714132	valid_1's auc: 0.714132
[500]	training's auc: 0.717775	training's auc: 0.717775	valid_1's auc: 0.716099	valid_1's auc: 0.716099
[600]	training's auc: 0.719513	training's auc: 0.719513	valid_1's auc: 0.717643	valid_1's auc: 0.717643
[700]	training's auc: 0.721069	training's auc: 0.721069	valid_1's auc: 0.719026	valid_1's auc: 0.719026
[800]	training's auc: 0.722358	training's auc: 0.722358	valid_1's auc: 0.720146	valid_1's auc: 0.720146
[900]	training's auc: 0.72356	training's auc: 0.72356	valid_1's auc: 0.721165	valid_1's auc: 0.721165
[100

In [43]:
pred_test = get_predicts(models, X_test)

In [44]:
log.save2("southman features", X_train, get_base_params(SEED), ["lightGbm"], validation_score)

In [45]:
log.get()

id         train_name  train_models  train_data_cnt  \
0  65  southman features  ["lightGbm"]         8921483   
1  64  southman features  ["lightGbm"]          100000   
2  63  southman features  ["lightGbm"]         8921483   
3  62  southman features  ["lightGbm"]         8921483   
4  61  southman features  ["lightGbm"]         8921483   
5  60  southman features  ["lightGbm"]         8921483   
6  59  southman features  ["lightGbm"]         8921483   
7  58  southman features  ["lightGbm"]         8921483   
8  57       southman lgb       ["lgb"]         8921483   
9  56       southman lgb       ["lgb"]         8921483   

                                              params     score  \
0  {"boosting_type": "gbdt", "objective": "binary...  0.725404   
1  {"boosting_type": "gbdt", "objective": "binary...  0.693786   
2  {"boosting_type": "gbdt", "objective": "binary...  0.727200   
3  {"boosting_type": "gbdt", "objective": "binary...  0.727863   
4  {"boosting_type": "gbdt", "objective": "binary...  0.741277   
5  {"boosting_type": "gbdt", "objective": "binary...  0.741277   
6  {"boosting_type": "gbdt", "objective": "binary...  0.720298   
7  {"boosting_type": "gbdt", "objective": "binary...  0.720298   
8  {"lgb__bagging_fraction": 0.8, "lgb__bagging_f...  0.649290   
9  {"lgb__bagging_fraction": 0.8, "lgb__bagging_f...  0.645404   

                                            features  features_cnt  \
0  ["EngineVersion", "AppVersion", "RtpStateBitfi...            46   
1  ["EngineVersion", "AppVersion", "RtpStateBitfi...            46   
2  ["EngineVersion", "AppVersion", "RtpStateBitfi...            64   
3  ["EngineVersion", "AppVersion", "RtpStateBitfi...            64   
4  ["ProductName", "EngineVersion", "AppVersion",...           116   
5  ["ProductName", "EngineVersion", "AppVersion",...           116   
6  ["ProductName", "EngineVersion", "AppVersion",...           116   
7  ["ProductName", "EngineVersion", "AppVersion",...           116   
8  ["ProductName", "EngineVersion", "AppVersion",...           102   
9  ["ProductName", "EngineVersion", "AppVersion",...           102   

                                        train_sample     insert_datetime  \
0  {"schema": {"fields":[{"name":"index","type":"... 2019-01-30 11:26:08   
1  {"schema": {"fields":[{"name":"index","type":"... 2019-01-29 18:50:12   
2  {"schema": {"fields":[{"name":"index","type":"... 2019-01-29 11:11:35   
3  {"schema": {"fields":[{"name":"index","type":"... 2019-01-28 04:43:36   
4  {"schema": {"fields":[{"name":"index","type":"... 2019-01-27 18:02:25   
5  {"schema": {"fields":[{"name":"index","type":"... 2019-01-27 18:01:35   
6  {"schema": {"fields":[{"name":"index","type":"... 2019-01-27 03:07:17   
7  {"schema": {"fields":[{"name":"index","type":"... 2019-01-27 03:01:56   
8  {"schema": {"fields":[{"name":"index","type":"... 2019-01-25 15:48:02   
9  {"schema": {"fields":[{"name":"index","type":"... 2019-01-25 14:35:47   

                                              source  
0  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
1  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
2  \n# coding: utf-8\n\n# In[64]:\n\n\n#!pip inst...  
3  \n# coding: utf-8\n\n# In[64]:\n\n\n#!pip inst...  
4  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
5  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
6  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
7  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
8  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...  
9  \n# coding: utf-8\n\n# In[ ]:\n\n\n#!pip insta...

In [46]:
%%time
submission = pd.DataFrame({'MachineIdentifier':ID_test, 'HasDetections':pred_test})
filename = 'submission_{:.6f}_{}_folds_{}_data.csv'.format(validation_score,dt.now().strftime('%Y-%m-%d-%H-%M'), len(models))
submission.to_csv('../result/single_{}'.format(filename), index=False)

Wall time: 29 s


In [ ]:
# submission = pd.DataFrame({'MachineIdentifier':ID_test, 'HasDetections':pred_test})
# submission.to_csv('../result/submission.csv', index=False)

In [ ]:
submission.head()